# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using

In [88]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import time

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. 

In [89]:
stocks = pd.read_csv('constituents.csv')
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [12]:
symbol = "AAPL"
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
data

{'beta': 1.1670842196914,
 'totalCash': 61555000000,
 'currentDebt': 145308000000,
 'revenue': 383285000000,
 'grossProfit': 169148000000,
 'totalRevenue': 383285000000,
 'EBITDA': 125820000000,
 'revenuePerShare': 24.64,
 'revenuePerEmployee': 2607380.95,
 'debtToEquity': 5.673462491552152,
 'profitMargin': 0.2530623426432028,
 'enterpriseValue': 3122290212640,
 'enterpriseValueToRevenue': 8.15,
 'priceToSales': 8.02,
 'priceToBook': 49.44,
 'forwardPERatio': 29.958553943814906,
 'pegRatio': 92.87203590336884,
 'peHigh': 11.929885083236506,
 'peLow': 8.01854304813428,
 'week52highDate': '2023-12-14',
 'week52lowDate': '2023-01-03',
 'putCallRatio': 0.4787638496632631,
 'companyName': 'Apple Inc',
 'marketcap': 3072757212640,
 'week52high': 199.62,
 'week52low': 123.15,
 'week52highSplitAdjustOnly': 199.62,
 'week52highDateSplitAdjustOnly': '2023-12-14',
 'week52lowSplitAdjustOnly': 124.17,
 'week52lowDateSplitAdjustOnly': '2023-01-03',
 'week52change': 0.48283411426611456,
 'sharesOut

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [13]:
data['year1ChangePercent']

0.48283411426611456

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [24]:
dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    quoteUrl = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(url).json()
    quoteData = requests.get(quoteUrl).json()
    symbols = symbol_string.split(',')
    for i in range(len(symbols)):
        name = symbols[i]
        price = quoteData[i]['latestPrice']
        oneYearChange = data[i]['year1ChangePercent']
        row = pd.DataFrame([[name, price, oneYearChange, 'N/A']], columns=my_columns)
        dataframe = pd.concat([dataframe, row], axis=0, ignore_index=True)
    time.sleep(0.4)

dataframe

/var/folders/13/07ny0fkn4db5gc851jjdw9x00000gn/T/ipykernel_13114/3828936445.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, row], axis=0, ignore_index=True)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,137.610,-0.077385,N/A
1,AAL,14.215,0.131148,N/A
2,AAP,60.490,-0.543075,N/A
3,AAPL,195.775,0.482834,N/A
4,ABBV,153.720,-0.001353,N/A
...,...,...,...,...
500,YUM,130.530,0.013067,N/A
501,ZBH,118.960,-0.047624,N/A
502,ZBRA,268.890,0.085409,N/A
503,ZION,43.285,-0.000766,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [28]:
dataframe.sort_values('One-Year Price Return', ascending = False, inplace=True)
dataframe = dataframe[:50]
dataframe.reset_index(inplace=True)
dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.920,2.281287,N/A
1,345,NVDA,498.980,1.951644,N/A
2,390,RCL,122.090,1.293952,N/A
3,370,PHM,101.710,1.229308,N/A
4,84,CCL,18.445,1.202367,N/A
5,31,AMD,138.810,1.127351,N/A
6,50,AVGO,1147.080,1.097355,N/A
7,114,CRM,265.370,1.039448,N/A
8,198,GE,123.320,1.021301,N/A
9,37,ANET,239.190,0.905224,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [32]:

class CheapPortfolioException(Exception):
    pass

def getPortfolioSize():
    val = 0
    while True:
        val = input("Enter the value of your portfolio: ")
        try:
            val = float(val)
            if val <= 0:
                raise CheapPortfolioException("That value is too small")
            return val
        except ValueError:
            print("That was not a number")
        except CheapPortfolioException:
            print("That number was too small")

portfolioSize = getPortfolioSize()
positionSize = portfolioSize/len(dataframe.index)

Enter the value of your portfolio:  123400


In [33]:

for i in range(len(dataframe)):
    dataframe.loc[i, "Number of Shares to Buy"] = math.floor(positionSize/dataframe.loc[i, 'Price'])

dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,79.920,2.281287,30
1,345,NVDA,498.980,1.951644,4
2,390,RCL,122.090,1.293952,20
3,370,PHM,101.710,1.229308,24
4,84,CCL,18.445,1.202367,133
5,31,AMD,138.810,1.127351,17
6,50,AVGO,1147.080,1.097355,2
7,114,CRM,265.370,1.039448,9
8,198,GE,123.320,1.021301,20
9,37,ANET,239.190,0.905224,10


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [131]:
hqmColumns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqmDataframe = pd.DataFrame(columns=hqmColumns)

for symbol_string in symbol_strings:
    url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    quoteUrl = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(url).json()
    quoteData = requests.get(quoteUrl).json()
    symbols = symbol_string.split(',')
    for i in range(len(symbols)):
        name = symbols[i]
        price = quoteData[i]['latestPrice']
        oneYearChange = data[i]['year1ChangePercent']
        sixMonthChange = data[i]['month6ChangePercent']
        threeMonthChange = data[i]['month3ChangePercent']
        oneMonthChange = data[i]['month1ChangePercent']
        row = pd.DataFrame([[name, price, 'N/A', oneYearChange, 'N/A', sixMonthChange, 'N/A', threeMonthChange, 'N/A', oneMonthChange, 'N/A', 'N/A']], columns=hqmColumns)
        hqmDataframe = pd.concat([hqmDataframe, row], axis=0, ignore_index=True)
    time.sleep(0.4)


/var/folders/13/07ny0fkn4db5gc851jjdw9x00000gn/T/ipykernel_13114/1386510234.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqmDataframe = pd.concat([hqmDataframe, row], axis=0, ignore_index=True)


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [132]:
timePeriods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in hqmDataframe.index:
    for period in timePeriods:
        columnString = f'{period} Price Return'
        percentile = stats.percentileofscore(hqmDataframe[columnString][~np.isnan(hqmDataframe[columnString])], hqmDataframe.loc[row, columnString])/100
        hqmDataframe.loc[row, f'{period} Return Percentile'] = percentile

hqmDataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.530,N/A,-0.077385,0.188525,0.133637,0.668033,0.182432,0.846311,0.208838,0.936475,N/A
1,AAL,14.250,N/A,0.131148,0.545082,-0.120752,0.122951,0.088655,0.563525,0.179007,0.893443,N/A
2,AAP,60.660,N/A,-0.543075,0.006148,-0.086234,0.161885,0.053617,0.459016,0.246175,0.963115,N/A
3,AAPL,195.900,N/A,0.482834,0.889344,0.074120,0.52459,0.131883,0.713115,0.041541,0.387295,N/A
4,ABBV,153.420,N/A,-0.001353,0.317623,0.134641,0.670082,0.022758,0.354508,0.113811,0.735656,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.470,N/A,0.013067,0.346311,-0.062967,0.196721,0.009833,0.317623,0.012443,0.227459,N/A
501,ZBH,118.800,N/A,-0.047624,0.241803,-0.173175,0.069672,-0.019473,0.237705,0.062864,0.483607,N/A
502,ZBRA,266.745,N/A,0.085409,0.463115,-0.043272,0.229508,0.074787,0.528689,0.239244,0.956967,N/A
503,ZION,43.040,N/A,-0.000766,0.321721,0.567887,0.991803,0.201883,0.893443,0.215414,0.944672,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [133]:
from statistics import mean

for row in hqmDataframe.index:
    momentumPercentiles = []
    for period in timePeriods:
        momentumPercentiles.append(hqmDataframe.loc[row, f'{period} Return Percentile'])
    hqmDataframe.loc[row, "HQM Score"] = mean(momentumPercentiles)
hqmDataframe
    


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,137.530,N/A,-0.077385,0.188525,0.133637,0.668033,0.182432,0.846311,0.208838,0.936475,0.659836
1,AAL,14.250,N/A,0.131148,0.545082,-0.120752,0.122951,0.088655,0.563525,0.179007,0.893443,0.53125
2,AAP,60.660,N/A,-0.543075,0.006148,-0.086234,0.161885,0.053617,0.459016,0.246175,0.963115,0.397541
3,AAPL,195.900,N/A,0.482834,0.889344,0.074120,0.52459,0.131883,0.713115,0.041541,0.387295,0.628586
4,ABBV,153.420,N/A,-0.001353,0.317623,0.134641,0.670082,0.022758,0.354508,0.113811,0.735656,0.519467
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.470,N/A,0.013067,0.346311,-0.062967,0.196721,0.009833,0.317623,0.012443,0.227459,0.272029
501,ZBH,118.800,N/A,-0.047624,0.241803,-0.173175,0.069672,-0.019473,0.237705,0.062864,0.483607,0.258197
502,ZBRA,266.745,N/A,0.085409,0.463115,-0.043272,0.229508,0.074787,0.528689,0.239244,0.956967,0.54457
503,ZION,43.040,N/A,-0.000766,0.321721,0.567887,0.991803,0.201883,0.893443,0.215414,0.944672,0.78791


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [134]:
hqmDataframe.sort_values("HQM Score", ascending=False, inplace=True)
hqmDataframe = hqmDataframe[:50]
hqmDataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
384,PVH,118.680,N/A,0.698618,0.961066,0.406253,0.977459,0.493638,0.997951,0.397172,1.0,0.984119
208,GPS,20.970,N/A,0.745791,0.969262,1.294383,1.0,0.929555,1.0,0.193838,0.913934,0.970799
370,PHM,102.000,N/A,1.229308,0.993852,0.397077,0.97541,0.343779,0.985656,0.160702,0.85041,0.951332
470,VNO,30.290,N/A,0.468131,0.877049,0.982934,0.997951,0.241102,0.92623,0.383397,0.997951,0.949795
462,URI,568.230,N/A,0.640196,0.942623,0.427538,0.979508,0.269167,0.954918,0.188698,0.905738,0.945697
132,DHI,148.600,N/A,0.694984,0.959016,0.295262,0.934426,0.338991,0.983607,0.174937,0.891393,0.942111
50,AVGO,1147.080,N/A,1.097355,0.987705,0.322447,0.95082,0.333709,0.981557,0.155472,0.840164,0.940061
390,RCL,121.100,N/A,1.293952,0.995902,0.285624,0.92623,0.257647,0.944672,0.160887,0.854508,0.930328
278,LEN,147.280,N/A,0.641074,0.944672,0.251708,0.881148,0.305130,0.977459,0.170915,0.881148,0.921107
413,SLG,45.760,N/A,0.529958,0.897541,0.851998,0.995902,0.162612,0.793033,0.380941,0.995902,0.920594


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [94]:
portfolioSize = getPortfolioSize()

Enter the value of your portfolio:  100000


In [135]:
positionSize = portfolioSize/len(hqmDataframe.index)
for i in hqmDataframe.index:
    hqmDataframe.loc[i, "Number of Shares to Buy"] = math.floor(positionSize/hqmDataframe.loc[i, "Price"])

hqmDataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
384,PVH,118.680,16,0.698618,0.961066,0.406253,0.977459,0.493638,0.997951,0.397172,1.0,0.984119
208,GPS,20.970,95,0.745791,0.969262,1.294383,1.0,0.929555,1.0,0.193838,0.913934,0.970799
370,PHM,102.000,19,1.229308,0.993852,0.397077,0.97541,0.343779,0.985656,0.160702,0.85041,0.951332
470,VNO,30.290,66,0.468131,0.877049,0.982934,0.997951,0.241102,0.92623,0.383397,0.997951,0.949795
462,URI,568.230,3,0.640196,0.942623,0.427538,0.979508,0.269167,0.954918,0.188698,0.905738,0.945697
132,DHI,148.600,13,0.694984,0.959016,0.295262,0.934426,0.338991,0.983607,0.174937,0.891393,0.942111
50,AVGO,1147.080,1,1.097355,0.987705,0.322447,0.95082,0.333709,0.981557,0.155472,0.840164,0.940061
390,RCL,121.100,16,1.293952,0.995902,0.285624,0.92623,0.257647,0.944672,0.160887,0.854508,0.930328
278,LEN,147.280,13,0.641074,0.944672,0.251708,0.881148,0.305130,0.977459,0.170915,0.881148,0.921107
413,SLG,45.760,43,0.529958,0.897541,0.851998,0.995902,0.162612,0.793033,0.380941,0.995902,0.920594


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [136]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqmDataframe.to_excel(writer, "Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [137]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [139]:
columnFormats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in columnFormats.keys():
    writer.sheets["Momentum Strategy"].set_column( f"{column}:{column}", 25, columnFormats[column][1])
    writer.sheets["Momentum Strategy"].write(f"{column}1", columnFormats[column][0], columnFormats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [140]:
writer.close()